### Plot faculty hiring distribution by city

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.autonotebook import tqdm
from postdocsalary.paths import DATA_DIR, ensure_dir
import urllib.request
pd.set_option('display.max_columns', None)
from postdocsalary.paths import save_fig, FIGURE_DIR

In [10]:
CBSA_df = pd.read_pickle(DATA_DIR/ 'CBSA_df_salary.pickle')

In [46]:
CBSA_df.sort_values(by='N_SALARIES', ascending=False)[:20]

,CBSA_NAME,RPP_all,total_hires,prop_hires,cum_hires,n_uni,n_grad,prop_grad,cum_grad,hiring_grad_disparity,MEDIAN_SALARY,N_SALARIES,MEDIAN_SALARY_UPPER,MEDIAN_SALARY_LOWER,REAL_MEDIAN_SALARY,REAL_MEDIAN_SALARY_UPPER,REAL_MEDIAN_SALARY_LOWER
6,"New York-Newark-Jersey City, NY-NJ-PA",125.7,854.0,0.061589,0.243545,5,1268.0,0.032211,0.129528,0.029379,55891.553082,2245,56013.506891,55328.923560,44464.242706,44561.262443,44016.645632
2,"Boston-Cambridge-Newton, MA-NH",115.5,1604.0,0.115679,0.115679,10,2325.0,0.059061,0.059061,0.056618,55474.915479,1683,55932.677979,55205.642506,48030.229852,48426.561021,47797.093079
24,"Houston-The Woodlands-Sugar Land, TX",101.7,121.0,0.008726,0.837011,2,390.0,0.009907,0.564218,-0.001181,51960.573989,1542,51964.730835,51871.600799,51092.009822,51096.097183,51004.523893
0,"Chicago-Naperville-Elgin, IL-IN-WI",102.8,919.0,0.066277,0.181956,6,1506.0,0.038256,0.097317,0.028021,53336.489989,1504,53830.036471,52850.139016,51883.745125,52363.848707,51410.641066
32,"Baltimore-Columbia-Towson, MD",105.8,130.0,0.009375,0.828285,2,582.0,0.014784,0.554311,-0.005409,55538.500043,1304,56013.506891,54987.657605,52493.856374,52942.823149,51973.211347
11,"Ann Arbor, MI",100.6,519.0,0.037430,0.470576,1,853.0,0.021668,0.266093,0.015761,51964.730835,1205,51964.730835,51964.730835,51654.802023,51654.802023,51654.802023
8,"San Francisco-Oakland-Berkeley, CA",134.5,671.0,0.048392,0.347685,1,911.0,0.023142,0.172052,0.025250,59110.348970,995,59110.348970,58413.470225,43948.214848,43948.214848,43430.089387
3,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",105.3,552.0,0.039810,0.433146,4,990.0,0.025149,0.244424,0.014661,54717.108962,949,55025.015351,54199.531014,51963.066441,52255.475167,51471.539425
16,"Los Angeles-Long Beach-Anaheim, CA",118.8,633.0,0.045651,0.393336,4,1859.0,0.047223,0.219276,-0.001572,57340.370036,788,57759.901346,56903.063787,48266.304744,48619.445577,47898.201841
14,"Durham-Chapel Hill, NC",94.8,455.0,0.032814,0.537646,2,866.0,0.021999,0.298074,0.010815,52766.604671,720,53100.089613,52418.861672,55660.975391,56012.752756,55294.157882


In [14]:
CBSA_df[:20]

,CBSA_NAME,RPP_all,total_hires,prop_hires,cum_hires,n_uni,n_grad,prop_grad,cum_grad,hiring_grad_disparity,MEDIAN_SALARY,N_SALARIES,MEDIAN_SALARY_UPPER,MEDIAN_SALARY_LOWER,REAL_MEDIAN_SALARY,REAL_MEDIAN_SALARY_UPPER,REAL_MEDIAN_SALARY_LOWER
2,"Boston-Cambridge-Newton, MA-NH",115.5,1604.0,0.115679,0.115679,10,2325.0,0.059061,0.059061,0.056618,55474.915479,1683,55932.677979,55205.642506,48030.229852,48426.561021,47797.093079
0,"Chicago-Naperville-Elgin, IL-IN-WI",102.8,919.0,0.066277,0.181956,6,1506.0,0.038256,0.097317,0.028021,53336.489989,1504,53830.036471,52850.139016,51883.745125,52363.848707,51410.641066
6,"New York-Newark-Jersey City, NY-NJ-PA",125.7,854.0,0.061589,0.243545,5,1268.0,0.032211,0.129528,0.029379,55891.553082,2245,56013.506891,55328.923560,44464.242706,44561.262443,44016.645632
5,"San Jose-Sunnyvale-Santa Clara, CA",126.7,773.0,0.055748,0.299293,1,763.0,0.019382,0.148910,0.036366,62476.603840,349,63319.925605,62352.688787,49310.658122,49976.263303,49212.856186
8,"San Francisco-Oakland-Berkeley, CA",134.5,671.0,0.048392,0.347685,1,911.0,0.023142,0.172052,0.025250,59110.348970,995,59110.348970,58413.470225,43948.214848,43948.214848,43430.089387
16,"Los Angeles-Long Beach-Anaheim, CA",118.8,633.0,0.045651,0.393336,4,1859.0,0.047223,0.219276,-0.001572,57340.370036,788,57759.901346,56903.063787,48266.304744,48619.445577,47898.201841
3,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",105.3,552.0,0.039810,0.433146,4,990.0,0.025149,0.244424,0.014661,54717.108962,949,55025.015351,54199.531014,51963.066441,52255.475167,51471.539425
11,"Ann Arbor, MI",100.6,519.0,0.037430,0.470576,1,853.0,0.021668,0.266093,0.015761,51964.730835,1205,51964.730835,51964.730835,51654.802023,51654.802023,51654.802023
1,"New Haven-Milford, CT",106.2,475.0,0.034256,0.504832,1,393.0,0.009983,0.276076,0.024273,53631.976988,583,54242.856101,53127.425417,50500.919951,51076.135688,50025.824310
14,"Durham-Chapel Hill, NC",94.8,455.0,0.032814,0.537646,2,866.0,0.021999,0.298074,0.010815,52766.604671,720,53100.089613,52418.861672,55660.975391,56012.752756,55294.157882


In [47]:
sample_list = [
    "Boston-Cambridge-Newton, MA-NH",
    "Chicago-Naperville-Elgin, IL-IN-WI",
    "New York-Newark-Jersey City, NY-NJ-PA",
    "San Francisco-Oakland-Berkeley, CA",
    #"Seattle-Tacoma-Bellevue, WA",
    "San Diego-Chula Vista-Carlsbad, CA",
    "Ann Arbor, MI",
    "Houston-The Woodlands-Sugar Land, TX",
    "Baltimore-Columbia-Towson, MD",
    "Durham-Chapel Hill, NC",
    "Pittsburgh, PA",
    "Portland-Vancouver-Hillsboro, OR-WA",
]

In [48]:
np.unique([i for i in CBSA_df.CBSA_NAME if 'Portland' in i])

array(['Portland-Vancouver-Hillsboro, OR-WA'], dtype='<U35')

In [49]:
CBSA_df[CBSA_df.CBSA_NAME.isin(sample_list)][
    ["CBSA_NAME", "RPP_all", "MEDIAN_SALARY", "REAL_MEDIAN_SALARY", "N_SALARIES", "prop_hires", "prop_grad"]
]

,CBSA_NAME,RPP_all,MEDIAN_SALARY,REAL_MEDIAN_SALARY,N_SALARIES,prop_hires,prop_grad
2,"Boston-Cambridge-Newton, MA-NH",115.5,55474.915479,48030.229852,1683,0.115679,0.059061
0,"Chicago-Naperville-Elgin, IL-IN-WI",102.8,53336.489989,51883.745125,1504,0.066277,0.038256
6,"New York-Newark-Jersey City, NY-NJ-PA",125.7,55891.553082,44464.242706,2245,0.061589,0.032211
8,"San Francisco-Oakland-Berkeley, CA",134.5,59110.348970,43948.214848,995,0.048392,0.023142
11,"Ann Arbor, MI",100.6,51964.730835,51654.802023,1205,0.037430,0.021668
14,"Durham-Chapel Hill, NC",94.8,52766.604671,55660.975391,720,0.032814,0.021999
4,"Pittsburgh, PA",92.4,51769.359077,56027.444889,611,0.023078,0.007468
32,"Baltimore-Columbia-Towson, MD",105.8,55538.500043,52493.856374,1304,0.009375,0.014784
24,"Houston-The Woodlands-Sugar Land, TX",101.7,51960.573989,51092.009822,1542,0.008726,0.009907
52,"San Diego-Chula Vista-Carlsbad, CA",117.9,56903.063787,48263.836970,616,0.005265,0.012320


In [44]:
59110/52761

1.1203350959989387

In [30]:
53760/ (94.8/100)

56708.86075949368

In [31]:
53760/ (134.5/100)

39970.260223048324

In [34]:
overall_median_salary = 53100.08961257449
overall_median_real_salary = 51848.669708466055

In [35]:
59110 / overall_median_salary

1.113180795574445

In [36]:
59110 - overall_median_salary

6009.910387425509

In [37]:
48030.229852 / overall_median_real_salary

0.9263541402713642